In [1]:
import warnings
warnings.filterwarnings("ignore")
from utils import *
import pickle
OPBG = [ 'código','age at diagnosis (months)',            
            'ntbc dosis mg/kg/day',
            'ntbc levels (dbs)',
            'sca (urine)',
            'methionine (plasma)',
            'tyrosine (plasma)',
            'phenylalanine (plasma)',
            'pt (sec)',
            'bili total',
            'gpt',
            'got',
            'ggt',
            'alkaline phosphatase',
            'alfa-fetoprotein','glicemia']


# OPBG = [ 'age at diagnosis (months)',    'código',   
#                 'ntbc dosis mg/kg/day',
#                 'ntbc levels (dbs)',
#                 'sca (urine)',
#                 'methionine dbs',
#                 'tyrosine dbs',
#                 'phenylalanine dbs',
#                 'methionine (plasma)',
#                 'tyrosine (plasma)',
#                 'phenylalanine (plasma)',
#                 'inr',
#                 'pt (sec)',
#                 'bili total',
#                 'gpt',
#                 'got',
#                 'ggt',
#                 'alkaline phosphatase',
#                 'alfa-fetoprotein',]
    
df1 = pd.read_csv('data/tirosinemia.csv').loc[:,OPBG]
df2 = pd.read_csv('data/tirosinemia_italia.csv').loc[:,OPBG]
def extract_transform(path, keep_cols, cols_rows_with_missing_values, new_binary_target):
    _ , spark_df = SparkDataProcessor().process_data(
        data_csv_path= path,
        )

    return prepare_spark_df(
        spark_df=spark_df,
        keep_cols=keep_cols,
        cols_rows_with_missing_values=cols_rows_with_missing_values,
        new_binary_target = new_binary_target,
    )
compare_dataframes(df1, df2)
df1.columns, df2.columns

Both CSV files have the same number of columns: 16
Column names are the same.
Column order is the same.
All common columns have the same data types.


(Index(['código', 'age at diagnosis (months)', 'ntbc dosis mg/kg/day',
        'ntbc levels (dbs)', 'sca (urine)', 'methionine (plasma)',
        'tyrosine (plasma)', 'phenylalanine (plasma)', 'pt (sec)', 'bili total',
        'gpt', 'got', 'ggt', 'alkaline phosphatase', 'alfa-fetoprotein',
        'glicemia'],
       dtype='object'),
 Index(['código', 'age at diagnosis (months)', 'ntbc dosis mg/kg/day',
        'ntbc levels (dbs)', 'sca (urine)', 'methionine (plasma)',
        'tyrosine (plasma)', 'phenylalanine (plasma)', 'pt (sec)', 'bili total',
        'gpt', 'got', 'ggt', 'alkaline phosphatase', 'alfa-fetoprotein',
        'glicemia'],
       dtype='object'))

In [2]:

binary_target = 'Alpha-Fet'
df_train, _ = extract_transform("data/tirosinemia.csv",                 OPBG, ['alfa-fetoprotein', 'sca (urine)'],binary_target)
df_train.drop('código', axis=1, inplace=True)
df_test, df_test_raw  = extract_transform("data/tirosinemia_italia.csv", OPBG, ['alfa-fetoprotein', 'sca (urine)'],binary_target)



23/06/07 17:59:53 WARN Utils: Your hostname, pop-os resolves to a loopback address: 127.0.1.1; using 100.64.221.100 instead (on interface tailscale0)
23/06/07 17:59:53 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/06/07 17:59:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/06/07 17:59:56 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [3]:

Rome_mask = df_test_raw['código'].str.startswith('R')

df_test_FLOR  = encode_and_convert (df_test[~Rome_mask].drop('código', axis=1).reset_index(inplace=False, drop=True))
df_test_ROME  = encode_and_convert (df_test[Rome_mask].drop('código', axis=1).reset_index(inplace=False, drop=True))
df_test_FLOR['Alpha-Fet'] = df_test_FLOR['Alpha-Fet'].astype('int8').astype('category')
df_test_ROME['Alpha-Fet'] = df_test_ROME['Alpha-Fet'].astype('int8').astype('category')


compare_dataframes(df_test_ROME, df_test_FLOR)


Both CSV files have the same number of columns: 15
Column names are the same.
Column order is the same.
All common columns have the same data types.


In [4]:


compare_dataframes(df_train, df_test_ROME)






Both CSV files have the same number of columns: 15
Column names are the same.
Column order is the same.
All common columns have the same data types.


In [5]:
df_train.to_parquet("data/train.parquet.gzip")
df_test_ROME.to_parquet("data/test.parquet.gzip")

In [6]:
ray.shutdown()
ray_df_train = ray.put(df_train)
ray_df_test  = ray.put(df_test_ROME)

2023-06-07 17:59:58,730	INFO worker.py:1616 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8266 


In [7]:
features : list[str] = df_train.columns.tolist()
features.remove(binary_target)
features

['age at diagnosis (months)',
 'ntbc dosis mg/kg/day',
 'ntbc levels (dbs)',
 'sca (urine)',
 'methionine (plasma)',
 'tyrosine (plasma)',
 'phenylalanine (plasma)',
 'pt (sec)',
 'bili total',
 'gpt',
 'got',
 'ggt',
 'alkaline phosphatase',
 'glicemia']

In [8]:
%%time

studies = make_multiple_studies(
    ray_df_train, 
    features = features,
    targets = [binary_target],
    n_trials=50,
    Independent_testset = False,
    Independent_testset_df = ray_df_test
)

studies = ray.get(studies)

(make_a_study pid=2212752) [I 2023-06-07 18:00:00,480] A new study created in memory with name: sca (urine) (Alpha-Fet)
(make_a_study pid=2212740) [I 2023-06-07 18:00:00,480] A new study created in memory with name: ntbc dosis mg/kg/day (Alpha-Fet)
(make_a_study pid=2212736) [I 2023-06-07 18:00:00,480] A new study created in memory with name: ntbc levels (dbs) (Alpha-Fet)
(make_a_study pid=2212754) [I 2023-06-07 18:00:00,466] A new study created in memory with name: age at diagnosis (months) (Alpha-Fet)
(make_a_study pid=2212740) [I 2023-06-07 18:00:15,256] Trial 5 finished with values: [0.5, 0.0] and parameters: {'seed': 853, 'max_depth': 7, 'eta': 0.4720516218071773, 'subsample': 0.23099763575945853, 'lambda': 0.208510711423589, 'alpha': 0.12134824946686984, 'scale_pos_weight': 0.07941396943580425, 'kfold_splits': 2}. 
(make_a_study pid=2212752) [I 2023-06-07 18:00:20,694] Trial 9 finished with values: [0.898989898989899, 0.0] and parameters: {'seed': 3494, 'max_depth': 2, 'eta': 0.0

KeyboardInterrupt: 

(make_a_study pid=2212752) [I 2023-06-07 18:00:40,460] A new study created in memory with name: methionine (plasma) (Alpha-Fet)
(make_a_study pid=2212736) [I 2023-06-07 18:00:40,990] A new study created in memory with name: phenylalanine (plasma) (Alpha-Fet)
(make_a_study pid=2212754) [I 2023-06-07 18:00:40,928] A new study created in memory with name: tyrosine (plasma) (Alpha-Fet)
(make_a_study pid=2212740) [I 2023-06-07 18:00:41,393] Trial 48 finished with values: [0.7752525252525253, 0.0] and parameters: {'seed': 9708, 'max_depth': 8, 'eta': 0.24726849527102523, 'subsample': 0.2044084772981529, 'lambda': 0.37561568445133375, 'alpha': 0.19121699847781137, 'scale_pos_weight': 0.6461076750864819, 'kfold_splits': 2}.  [repeated 46x across cluster]
(make_a_study pid=2212740) [I 2023-06-07 18:00:41,783] A new study created in memory with name: pt (sec) (Alpha-Fet)
(make_a_study pid=2212752) [I 2023-06-07 18:00:56,889] Trial 2 finished with values: [0.5744949494949496, 0.29988375306129456]

In [ ]:
with open('chile.pickle', 'wb') as handle:
    pickle.dump(studies, handle, protocol=pickle.HIGHEST_PROTOCOL)    